# How to apply schema to spark dataframes

In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import pyspark
from pyspark.sql import SparkSession

## Start local spark session

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/02 16:43:48 WARN Utils: Your hostname, ROG-STRIX resolves to a loopback address: 127.0.1.1; using 172.27.115.120 instead (on interface eth0)
23/03/02 16:43:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 16:43:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import pandas as pd

In [5]:
from pyspark.sql import types

## Import data into spark

In [6]:
df_fhv = spark.read \
    .option("header", "true") \
    .csv("data/raw/fhv/2021/01/")

In [7]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2021-01-01 00:27:00|2021-01-01 00:44:00|        null|        null|   null|                B00009|
|              B00009|2021-01-01 00:50:00|2021-01-01 01:07:00|        null|        null|   null|                B00009|
|              B00013|2021-01-01 00:01:00|2021-01-01 01:51:00|        null|        null|   null|                B00013|
|              B00037|2021-01-01 00:13:09|2021-01-01 00:21:26|        null|          72|   null|                B00037|
|              B00037|2021-01-01 00:38:31|2021-01-01 00:53:44|        null|          61|   null|                B00037|
|              B00037|2021-01-01 00:59:0

In [8]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



> As you can see, spark imported the data but was unable to determine the types of the features.

## Import the same data into a Pandas DataFrame

In [9]:
df_fhv_pd = pd.read_csv("data/raw/fhv/2021/01/fhv_tripdata_2021-01.csv.gz", nrows=1000)
df_fhv_pd

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,NaN,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,NaN,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,NaN,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,NaN,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,NaN,B00037
...,...,...,...,...,...,...,...
995,B00009,2021-01-01 01:57:00,2021-01-01 02:41:00,NaN,NaN,NaN,B00009
996,B00009,2021-01-01 01:45:00,2021-01-01 01:56:00,NaN,NaN,NaN,B00009
997,B00009,2021-01-01 01:06:00,2021-01-01 01:18:00,NaN,NaN,NaN,B00009
998,B00037,2021-01-01 01:51:27,2021-01-01 01:55:51,NaN,85.0,NaN,B00037


In [10]:
df_fhv_pd.dtypes

dispatching_base_num       object
pickup_datetime            object
dropoff_datetime           object
PULocationID              float64
DOLocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

> As you can tell, Pandas did a better job of infering the types from the data

Remove extra spacing after some of the entries in Affiliated base number.

In [11]:
df_fhv_pd.Affiliated_base_number = df_fhv_pd.Affiliated_base_number.apply(lambda x: str(x).strip())

## Import the DataFrame into Spark and extract it's schema

In [12]:
spark.createDataFrame(df_fhv_pd).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', DoubleType(), True), StructField('DOLocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

The default schema isn't the best so I've modified some of the fields.

```python
fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.BooleanType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])
```

## Declare schemas

In [13]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.BooleanType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

### Green Taxi Data

In [14]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(6) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4
processing data for 2020/5
processing data for 2020/6
processing data for 2020/7
processing data for 2020/8
processing data for 2020/9
processing data for 2020/10
processing data for 2020/11
processing data for 2020/12


In [15]:
year = 2021

for month in range(1, 8):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(6) \
        .write.parquet(output_path)

processing data for 2021/1
processing data for 2021/2
processing data for 2021/3
processing data for 2021/4
processing data for 2021/5
processing data for 2021/6
processing data for 2021/7


### Yellow Taxi Data

In [16]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(6) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [17]:
year = 2021

for month in range(1, 8):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(6) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7


### FHV Taxi Data

In [18]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/fhv/{year}/{month:02d}/'
    output_path = f'data/pq/fhv/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(fhv_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(6) \
        .write.parquet(output_path)
#         .write.parquet(output_path, mode='overwrite')

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [19]:
year = 2021

for month in range(1, 8):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/fhv/{year}/{month:02d}/'
    output_path = f'data/pq/fhv/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(fhv_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(6) \
        .write.parquet(output_path)
#         .write.parquet(output_path, mode='overwrite')

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7
